In [1]:
from USTs import USTs
from DataFetcher import DataFetcher
import datetime
from scipy.optimize import newton
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests

In [2]:
CUSIPs = ['912810RB6', '912810SR0']
START = datetime.date(2025, 3, 1)
END = datetime.date(2025, 6, 30)
# USTs(None, None).get_cusip_timeseries(cusips, datetime.date(2025, 1, 1), datetime.date(2025, 6, 30))

In [3]:
price_data = DataFetcher().fetch_cusip_timeseries(CUSIPs, START, END)
ytm_timeseries = price_data.copy()
        
if len(CUSIPs) == 2:
    if ytm_timeseries.shape[1] < 2:
        print("wrong shape")
    else:
        ytm_timeseries['Spread'] = ytm_timeseries.iloc[:, 1] - ytm_timeseries.iloc[:, 0]
ytm_timeseries

,912810SR0,912810RB6,Spread
Date,,,
2025-03-03,63.90625,79.75000,15.84375
2025-03-04,63.28125,78.96875,15.68750
2025-03-05,62.81250,78.43750,15.62500
2025-03-06,62.59375,78.21875,15.62500
2025-03-07,62.40625,77.93750,15.53125
...,...,...,...
2025-06-24,61.65625,76.53125,14.87500
2025-06-25,61.68750,76.53125,14.84375
2025-06-26,61.81250,76.75000,14.93750


In [4]:
date_range = pd.date_range(start=START, end=END).to_list()
all_data = dict()
        
for date in date_range:
    date_price_info = dict()
    payload = {
         "priceDate.month": date.month,
         "priceDate.day": date.day,
         "priceDate.year": date.year,
         "submit": "Show Prices"
         }
    response = requests.post(url="https://www.treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate", data=payload, timeout=30)
    response.raise_for_status()

    output = BeautifulSoup(response.text, "html.parser")
    table = output.find("table", class_="data1")
    data = table.find_all("tr")[1:]
    if data:
        for tr in data:
            data_row = [td.get_text(strip=True) for td in tr.find_all("td")]
            if data_row[0] in CUSIPs: date_price_info[data_row[0]] = data_row[7]
        all_data[date] = date_price_info
        
prices_df = pd.DataFrame(data=all_data).transpose().rename_axis("Date").dropna(axis=0)
for col in prices_df.columns:
    prices_df[col] = pd.to_numeric(prices_df[col])
prices_df

KeyboardInterrupt: 

In [ ]:
PRICE = 89.9063
ISSUE = datetime.datetime(2009, 2, 17)
RATE = 3.500
MATURITY = datetime.datetime(2039, 2, 15)
SETTLEMENT = datetime.datetime(2025, 6, 27)

In [ ]:
USTs(None, None).get_coupon_ytm(PRICE, ISSUE, MATURITY, SETTLEMENT, RATE, dirty=False)

4.496087

In [ ]:
import rateslib as rl
bond = rl.FixedRateBond(
    effective=ISSUE,
    termination=MATURITY,
    fixed_rate = RATE,
    spec="us_gb"  # US Government Bond
)
rateslib = bond.ytm(price=PRICE, settlement=SETTLEMENT)
rateslib

4.497986422899381